
# TP2 — Redes Neuronales con Keras (Adult Income)

> **Alumno/a:** _Completar_  
> **Materia:** _Completar_  
> **Fecha:** 2025-11-10

Este notebook implementa una red neuronal con **Keras (TensorFlow)** para el dataset **Adult Income** (el mismo del TP1).  
Incluye:
- Preprocesamiento (one‑hot encoding + escalado de numéricas)
- División en **train / validación / test**
- **3 sets** distintos de hiperparámetros (profundidad, neuronas, dropout, learning rate)
- Entrenamiento y evaluación con **accuracy** (y matriz de confusión opcional)
- Selección del mejor set por **accuracy en validación**
- Evaluación final en **test**
- Espacio para **comparar** con lo obtenido en el TP1

> Nota: Si no tenés el CSV local, el notebook intentará descargarlo automáticamente de UCI/OpenML.


In [ ]:

# ==== Setup ====
import os
import sys
import math
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(tf.__version__)



## 1) Carga de datos

Usamos el dataset **Adult Income**. El código intenta:
1. Cargar `adult.csv` o `adult.data` locales (si existen).
2. Descargar desde UCI / OpenML si no están (requiere internet).


In [ ]:

def try_load_adult():
    # 1) CSV/ARFF locales conocidos
    local_candidates = [
        "adult.csv",
        "adult.data",
        "adult/adult.csv",
        "adult/adult.data"
    ]
    for p in local_candidates:
        if os.path.exists(p):
            if p.endswith(".csv"):
                df = pd.read_csv(p)
            else:
                # adult.data separado por coma sin encabezados
                cols = [
                    "age","workclass","fnlwgt","education","education-num","marital-status",
                    "occupation","relationship","race","sex","capital-gain","capital-loss",
                    "hours-per-week","native-country","income"
                ]
                df = pd.read_csv(p, header=None, names=cols, na_values="?\s*", skipinitialspace=True)
            print(f"Cargado local: {p}")
            return df

    # 2) Intento UCI
    try:
        uci_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
        cols = [
            "age","workclass","fnlwgt","education","education-num","marital-status",
            "occupation","relationship","race","sex","capital-gain","capital-loss",
            "hours-per-week","native-country","income"
        ]
        df = pd.read_csv(uci_url, header=None, names=cols, na_values="?\s*", skipinitialspace=True)
        print("Descargado desde UCI.")
        return df
    except Exception as e:
        print("No se pudo descargar desde UCI:", e)

    # 3) Intento OpenML
    try:
        from sklearn.datasets import fetch_openml
        adult = fetch_openml('adult', version=2, as_frame=True)
        df = adult.frame
        # Ajuste de nombre/etiqueta si viniera distinto
        if 'class' in df.columns and 'income' not in df.columns:
            df = df.rename(columns={'class': 'income'})
        print("Descargado desde OpenML.")
        return df
    except Exception as e:
        print("No se pudo descargar desde OpenML:", e)

    raise FileNotFoundError("No se encontró dataset local ni conexión para descargar. "
                            "Colocá un 'adult.csv' junto al notebook y reejecutá.")

df = try_load_adult()
print(df.head())
print(df.shape)



## 2) Limpieza y preparación

- Eliminamos filas con `NaN` (provenientes de valores `'?'`).
- Definimos **features** y **target** (`income`: `>50K` vs `<=50K`).


In [ ]:

# Uniformar nombre de columna target si fuese necesario
if 'income' not in df.columns:
    # intentar inferir
    cand = [c for c in df.columns if c.lower() in ('income','class','target','label')]
    assert len(cand) == 1, f"No se encontró target unívoco, columnas: {df.columns}"
    df = df.rename(columns={cand[0]: 'income'})

# Drop NAs
df = df.dropna().reset_index(drop=True)

# Target binario (str -> 0/1)
df['income'] = df['income'].astype(str).str.strip()
y = (df['income'] == '>50K').astype(int).values  # 1 si >50K, 0 si <=50K
X = df.drop(columns=['income'])

# Definir columnas numéricas y categóricas
numeric_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()

print("Numéricas:", numeric_cols)
print("Categóricas:", categorical_cols)



## 3) Split: train / validación / test

- Primero separamos **train (60%)** y **temp (40%)**  
- Luego dividimos **temp** en **validación (20%)** y **test (20%)** (del total)


In [ ]:

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.40, random_state=SEED, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=SEED, stratify=y_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)



## 4) Pipeline de preprocesamiento

- `OneHotEncoder` para categóricas (sin `drop` para mantener toda la info).
- `StandardScaler` para numéricas.
- Armamos un `ColumnTransformer` y lo **ajustamos solo con train**.


In [ ]:

preprocess = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
])

X_train_pp = preprocess.fit_transform(X_train)
X_val_pp   = preprocess.transform(X_val)
X_test_pp  = preprocess.transform(X_test)

input_dim = X_train_pp.shape[1]
input_dim



## 5) Definición de modelos (3 sets de hiperparámetros)

Creamos una función `build_model(**h)` y tres diccionarios de hiperparámetros:
- **Set A (chico):** 1 capa oculta (64), `dropout=0.0`, `lr=1e-3`
- **Set B (medio):** 2 capas (128, 64), `dropout=0.2`, `lr=5e-4`
- **Set C (grande):** 3 capas (256, 128, 64), `dropout=0.3`, `lr=3e-4`


In [ ]:

def build_model(input_dim, hidden_layers=(64,), dropout=0.0, lr=1e-3, l2_reg=0.0):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    for units in hidden_layers:
        model.add(layers.Dense(units, activation='relu',
                               kernel_regularizer=keras.regularizers.l2(l2_reg) if l2_reg>0 else None))
        if dropout > 0:
            model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

hyperparams_list = [
    {"name": "Set A (chico)",  "hidden_layers": (64,),              "dropout": 0.0, "lr": 1e-3, "l2_reg": 0.0, "epochs": 20, "batch_size": 256},
    {"name": "Set B (medio)",  "hidden_layers": (128, 64),          "dropout": 0.2, "lr": 5e-4, "l2_reg": 1e-5, "epochs": 25, "batch_size": 256},
    {"name": "Set C (grande)", "hidden_layers": (256, 128, 64),     "dropout": 0.3, "lr": 3e-4, "l2_reg": 1e-5, "epochs": 30, "batch_size": 512},
]
hyperparams_list



## 6) Entrenamiento y evaluación en **validación**

Entrenamos cada set con `EarlyStopping` monitorizando `val_loss`.  
Guardamos accuracy en validación y seleccionamos el mejor.


In [ ]:

histories = {}
val_scores = []

for h in hyperparams_list:
    print("\n==== Entrenando:", h["name"], "====")
    model = build_model(
        input_dim=input_dim,
        hidden_layers=h["hidden_layers"],
        dropout=h["dropout"],
        lr=h["lr"],
        l2_reg=h["l2_reg"]
    )
    cb = [
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
    history = model.fit(
        X_train_pp, y_train,
        validation_data=(X_val_pp, y_val),
        epochs=h["epochs"],
        batch_size=h["batch_size"],
        verbose=1,
        callbacks=cb
    )
    histories[h["name"]] = history.history

    # Eval en validación
    val_pred = (model.predict(X_val_pp) >= 0.5).astype(int).ravel()
    val_acc = accuracy_score(y_val, val_pred)
    val_scores.append((h["name"], val_acc, model))
    print(f"Accuracy de validación ({h['name']}): {val_acc:.4f}")

# Ordenar por accuracy desc
val_scores.sort(key=lambda x: x[1], reverse=True)
best_name, best_val_acc, best_model = val_scores[0]
print("\nMejor set por validación:", best_name, "— acc:", round(best_val_acc, 4))



### Curvas de entrenamiento (loss y accuracy)


In [ ]:

# Una figura por métrica, sin estilos de color específicos
plt.figure()
for name, hist in histories.items():
    plt.plot(hist['loss'], label=f"{name} - train")
    plt.plot(hist['val_loss'], label=f"{name} - val")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Pérdida (train vs val)")
plt.legend()
plt.show()

plt.figure()
for name, hist in histories.items():
    plt.plot(hist['accuracy'], label=f"{name} - train")
    plt.plot(hist['val_accuracy'], label=f"{name} - val")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy (train vs val)")
plt.legend()
plt.show()



## 7) Evaluación en **test** (mejor set)


In [ ]:

test_pred = (best_model.predict(X_test_pp) >= 0.5).astype(int).ravel()
test_acc = accuracy_score(y_test, test_pred)
print(f"Accuracy en test con '{best_name}': {test_acc:.4f}")

# Matriz de confusión y reporte (opcional)
cm = confusion_matrix(y_test, test_pred)
print("\nMatriz de confusión:\n", cm)
print("\nReporte de clasificación:\n", classification_report(y_test, test_pred, digits=4))



## 8) Comparación con TP1

**Completar con los resultados de TP1** (por ejemplo):
- Regresión logística: _accuracy = ..._
- Árbol de decisión: _accuracy = ..._
- (Otros si usaste)

**Resumen comparativo:** _Escribir análisis: ¿mejoró Keras? ¿sobreajuste? ¿tiempos? ¿sensibilidad a hiperparámetros?_



## (Opcional) Guardado de artefactos

Guardamos el **preprocesador** (`sklearn`) y el **modelo** (`Keras`) para uso futuro.


In [ ]:

import joblib
outdir = pathlib.Path("artefactos_modelo")
outdir.mkdir(exist_ok=True, parents=True)

joblib.dump(preprocess, outdir / "preprocess.joblib")
best_model.save(outdir / "keras_best_model.keras")
print("Guardado en:", outdir.resolve())
